### 2025 Predictions

In [2]:
# =============================================================================
# DATASET CONFIGURATION - Easy switching between dataset subsets
# =============================================================================

import sys
from pathlib import Path
sys.path.insert(0, str(Path('../').resolve()))

from dataset_config import (
    DEFAULT_CONFIG, get_dataset_config, get_dataset_path, get_config_summary,
    use_full_dataset, use_english_only, use_major_studios, use_english_major
)

# =============================================================================
# CHOOSE YOUR DATASET SCOPE - Uncomment one line to switch
# =============================================================================

# CURRENT_CONFIG = use_full_dataset()                    # All studios, all languages (2010-2026)
# CURRENT_CONFIG = use_english_only(2010)                # English only (2010-2026)
CURRENT_CONFIG = use_english_only(2015)                # English only (2015-2026)
# CURRENT_CONFIG = use_major_studios(2010)               # Major studios only (2010-2026)
# CURRENT_CONFIG = use_major_studios(2015)               # Major studios only (2015-2026)
# CURRENT_CONFIG = use_english_major(2010)               # English + Major studios (2010-2026)
# CURRENT_CONFIG = use_english_major(2015)               # English + Major studios (2015-2026)

# CURRENT_CONFIG = DEFAULT_CONFIG  # Use default (full dataset)

print("🎯 DATASET CONFIGURATION:")
print("=" * 50)
print(get_config_summary(CURRENT_CONFIG))
print("\n💡 To change scope, uncomment one of the CURRENT_CONFIG lines above and re-run this cell")

🎯 DATASET CONFIGURATION:
📊 Dataset Configuration: English Only
   Description: English movies only
   Year range: 2015-2026
   Training file: dataset_domestic_processed_english_2015_2026.csv
   Full file: dataset_domestic_processed_english_2015_2026.csv


💡 To change scope, uncomment one of the CURRENT_CONFIG lines above and re-run this cell


In [3]:
# Setup imports and MLflow tracking
from pathlib import Path
import json

import numpy as np
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient

from movie_lists import normalize_domestic_titles
from model_utils import (
    prepare_features,
    compute_ranking_metrics,
    get_top10_predictions,
)

pd.options.display.float_format = lambda x: f'{x:,.2f}'
np.set_printoptions(suppress=True)

TRACKING_DIR_CANDIDATES = [
    Path('../mlruns'),
    Path('mlruns'),
]
tracking_path = None
for candidate in TRACKING_DIR_CANDIDATES:
    if candidate.exists():
        tracking_path = candidate.resolve()
        break
if tracking_path is None:
    raise FileNotFoundError('Unable to locate mlruns tracking directory')

mlflow.set_tracking_uri(tracking_path.as_uri())


In [4]:
# Dataset loading with configuration system
def load_dataset(training=False):
    """Load the configured dataset subset."""
    import pandas as pd
    from movie_lists import normalize_domestic_titles

    dataset_path = get_dataset_path(training=training, config=CURRENT_CONFIG)
    dataset_config = get_dataset_config(CURRENT_CONFIG)

    print(f"📁 Loading dataset: {dataset_path}")
    
    # Load data
    df = pd.read_csv(dataset_path)

    # Apply additional filtering if needed (for english_major scope)
    if dataset_config['scope'] == 'english_major':
        if 'is_major_studio' in df.columns:
            original_len = len(df)
            df = df[df['is_major_studio'] == 1].copy()
            print(f"   Filtered to major studios: {len(df):,} movies (removed {original_len - len(df):,})")

    # Normalize titles
    df = normalize_domestic_titles(df)

    print(f"   ✅ Loaded {len(df):,} movies")
    if 'release_year' in df.columns:
        print(f"   Year range: {df['release_year'].min()}-{df['release_year'].max()}")

        # Show breakdown by time period
        training_count = len(df[df['release_year'] <= 2023])
        test_2024_count = len(df[df['release_year'] == 2024])
        eval_2025_count = len(df[df['release_year'] == 2025])
        pred_2026_count = len(df[df['release_year'] == 2026])

        print(f"   Training (≤2023): {training_count:,} movies")
        if test_2024_count > 0:
            print(f"   Testing (2024): {test_2024_count:,} movies")
        if eval_2025_count > 0:
            print(f"   Evaluation (2025): {eval_2025_count:,} movies")
        if pred_2026_count > 0:
            print(f"   Prediction (2026): {pred_2026_count:,} movies")

    return df

# Legacy helper utilities for dataset resolution (keep for compatibility)
DATA_DIR = Path('../data')
DEFAULT_DATASET_CANDIDATES = [
    DATA_DIR / 'dataset_domestic_processed.csv',          # FULL dataset with all years
    DATA_DIR / 'dataset_domestic_processed_modeling.csv', # Training-only dataset (fallback)
]

def _first_existing(paths):
    for candidate in paths:
        if candidate.exists():
            return candidate
    return None

DEFAULT_DATASET_PATH = _first_existing(DEFAULT_DATASET_CANDIDATES)

SUBSET_PATHS = {
    ('english_only', 'all_studios', 2010): DATA_DIR / 'dataset_domestic_processed_english_2010_2026.csv',
    ('english_only', 'all_studios', 2015): DATA_DIR / 'dataset_domestic_processed_english_2015_2026.csv',
    ('all_languages', 'major_only', 2010): DATA_DIR / 'dataset_domestic_processed_major_2010_2026.csv',
    ('all_languages', 'major_only', 2015): DATA_DIR / 'dataset_domestic_processed_major_2015_2026.csv',
}

def infer_year_floor(params):
    if not params:
        return 2015
    getter = params.get if hasattr(params, 'get') else (lambda key: None)
    for key in ('train_year_min', 'train_start_year', 'training_year_start'):
        value = getter(key)
        if value is None:
            continue
        try:
            return int(value)
        except ValueError:
            continue
    return 2015

def resolve_dataset_path(studio_scope, language_scope, params=None):
    """Legacy function - now uses configuration system as fallback"""
    try:
        # Try to use the configuration system first
        return get_dataset_path(training=False, config=CURRENT_CONFIG)
    except:
        # Fallback to legacy logic
        lang_scope = (language_scope or 'all_languages').lower()
        studio_scope = (studio_scope or 'all_studios').lower()
        params = params or {}

        year_floor = infer_year_floor(params)
        year_pref = [2015, 2010] if year_floor and year_floor > 2010 else [2010, 2015]

        candidates = []
        if lang_scope.startswith('english'):
            for yr in [2010, 2015]:
                candidates.append(('english_only', 'all_studios', yr))
        if studio_scope == 'major_only':
            for yr in year_pref:
                candidates.append(('all_languages', 'major_only', yr))
        for yr in year_pref:
            candidates.append((
                lang_scope if lang_scope in ('english_only', 'all_languages') else 'all_languages',
                studio_scope if studio_scope in ('all_studios', 'major_only') else 'all_studios',
                yr
            ))

        seen = set()
        ordered_candidates = []
        for key in candidates:
            if key not in seen:
                seen.add(key)
                ordered_candidates.append(key)

        for key in ordered_candidates:
            path = SUBSET_PATHS.get(key)
            if path and path.exists():
                return path

        if DEFAULT_DATASET_PATH is not None:
            return DEFAULT_DATASET_PATH
        raise FileNotFoundError('No suitable dataset found for the requested scope.')

def load_prepared_dataset(dataset_path):
    """Legacy function - now uses configuration system"""
    from model_utils import prepare_features
    
    df_prepared = load_dataset(training=False)
    df_prepared, feature_cols_prepared, target_name = prepare_features(
        df_prepared,
        target='revenue_domestic',
        verbose=False,
    )
    print(f'Loaded dataset: {dataset_path} ({len(df_prepared)} rows)')
    print(f'   Feature columns: {len(feature_cols_prepared)}')
    print(f'   Target column: {target_name}')
    return df_prepared, feature_cols_prepared, target_name

In [5]:
# Locate the best MLflow run (Recall@10 priority)
experiment_name = 'box_office_modeling'
client = MlflowClient()
best_run = None
best_run_params = {}
best_run_dataset_path = None
mlflow_results = pd.DataFrame()

experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiments = client.search_experiments()
    finished = []
    for exp in experiments:
        runs = mlflow.search_runs(
            [exp.experiment_id],
            filter_string="attributes.status = 'FINISHED'",
            max_results=1,
        )
        if not runs.empty:
            finished.append(exp)
    if finished:
        experiment = max(finished, key=lambda exp: getattr(exp, 'last_update_time', 0) or 0)
        print(f"Default experiment missing; using '{experiment.name}' instead")

if experiment is not None:
    runs_df = mlflow.search_runs(
        [experiment.experiment_id],
        filter_string="attributes.status = 'FINISHED'",
        order_by=['metrics.rmse ASC'],
    )
    if not runs_df.empty:
        rename_map = {
            'tags.mlflow.runName': 'run_name',
            'params.model': 'model',
            'params.strategy': 'strategy',
            'params.data_scope_studios': 'data_scope_studios',
            'params.data_scope_language': 'data_scope_language',
            'metrics.rmse': 'rmse',
            'metrics.recall_at_10': 'recall_at_10',
            'metrics.precision_at_10': 'precision_at_10',
            'metrics.ndcg_at_10': 'ndcg_at_10',
            'metrics.r2': 'r2',
            'metrics.mae': 'mae',
            'metrics.mape': 'mape',
            'metrics.spearman_corr': 'spearman_corr',
            'metrics.kendall_corr': 'kendall_corr',
        }
        keep_cols = [c for c in rename_map if c in runs_df.columns] + ['run_id']
        mlflow_results = runs_df[keep_cols].rename(columns=rename_map)
        for col in ('rmse', 'recall_at_10', 'precision_at_10', 'ndcg_at_10', 'r2', 'mae', 'mape'):
            if col in mlflow_results.columns:
                mlflow_results[col] = pd.to_numeric(mlflow_results[col], errors='coerce')
        if 'recall_at_10' in mlflow_results.columns and mlflow_results['recall_at_10'].notna().any():
            mlflow_results = mlflow_results.sort_values(['recall_at_10', 'rmse'], ascending=[False, True])
        elif 'rmse' in mlflow_results.columns:
            mlflow_results = mlflow_results.sort_values('rmse')
        mlflow_results = mlflow_results.reset_index(drop=True)
        if not mlflow_results.empty:
            best_run = mlflow_results.iloc[0]
            run_id = best_run['run_id']
            try:
                run_info = client.get_run(run_id)
                best_run_params = run_info.data.params
            except Exception as exc:
                print(f"Unable to fetch parameters for run {run_id}: {exc}")
                best_run_params = {}
            try:
                best_run_dataset_path = resolve_dataset_path(
                    best_run.get('data_scope_studios'),
                    best_run.get('data_scope_language'),
                    best_run_params,
                )
                print(f"Dataset selected: {best_run_dataset_path}")
            except Exception as exc:
                print(f"Unable to resolve dataset path: {exc}")
                best_run_dataset_path = DEFAULT_DATASET_PATH
            recall_text = f"Recall@10 {best_run.get('recall_at_10', float('nan')):.2%}"
            rmse_text = f"RMSE ${best_run.get('rmse', float('nan')):,.0f}"
            print(f"Best run: {best_run.get('run_name')} ({recall_text}, {rmse_text})")
    else:
        print('No finished MLflow runs found in the experiment.')
else:
    print('No MLflow experiment with finished runs is available.')


Dataset selected: ../data/dataset_domestic_processed_english_2015_2026.csv
Best run: LightGBM | No Pandemic Era (Recall@10 80.00%, RMSE $78,937,117)


In [6]:
# Evaluate the best model on 2025 data
if best_run is None or best_run_dataset_path is None:
    print('Cannot evaluate because no best MLflow run was identified.')
else:
    df_prepared, feature_cols, target = load_prepared_dataset(best_run_dataset_path)
    test_2025 = df_prepared[df_prepared['release_year'] == 2025].copy()
    if test_2025.empty:
        print('No 2025 rows available in the selected dataset.')
    else:
        run_id = best_run['run_id']
        try:
            best_model = mlflow.sklearn.load_model(f'runs:/{run_id}/model')
        except Exception:
            best_model = mlflow.pyfunc.load_model(f'runs:/{run_id}/model')
        if best_model is None:
            print(f"Unable to load model artifacts for run {run_id}")
        else:
            X_test = test_2025[feature_cols]
            preds_log = best_model.predict(X_test)
            preds = np.expm1(preds_log)
            test_2025['predicted_revenue'] = preds

            if target in test_2025.columns and test_2025[target].notna().any():
                rmse = float(np.sqrt(np.mean((test_2025[target] - preds) ** 2)))
                mae = float(np.mean(np.abs(test_2025[target] - preds)))
                with np.errstate(divide='ignore', invalid='ignore'):
                    mape = float(np.nanmean(np.abs((test_2025[target] - preds) / test_2025[target])) * 100)
                if test_2025[target].nunique() > 1:
                    ss_res = np.sum((test_2025[target] - preds) ** 2)
                    ss_tot = np.sum((test_2025[target] - test_2025[target].mean()) ** 2)
                    r2 = float(1 - ss_res / ss_tot) if ss_tot > 0 else float('nan')
                else:
                    r2 = float('nan')
            else:
                rmse = mae = mape = r2 = float('nan')

            ranking_metrics = compute_ranking_metrics(
                test_2025,
                preds,
                target_col=target,
                title_col='title',
                k=10,
            )

            print('2025 evaluation snapshot:')
            if not np.isnan(rmse):
                print(f'   RMSE: ${rmse:,.0f}')
                print(f'   MAE:  ${mae:,.0f}')
                print(f'   MAPE: {mape:.1f}%')
                print(f'   R^2:  {r2:.3f}')
            else:
                print('   Actual revenue not available for full error metrics.')
            if ranking_metrics:
                if 'recall_at_10' in ranking_metrics:
                    print(f"   Recall@10: {ranking_metrics['recall_at_10']:.2%}")
                if 'precision_at_10' in ranking_metrics:
                    print(f"   Precision@10: {ranking_metrics['precision_at_10']:.2%}")
                if 'ndcg_at_10' in ranking_metrics:
                    print(f"   NDCG@10: {ranking_metrics['ndcg_at_10']:.3f}")

            top10_2025 = get_top10_predictions(best_model, df_prepared, 2025, feature_cols)
            if top10_2025 is not None:
                display(top10_2025)
                predicted_top10 = top10_2025['title'].tolist()
                actual_available = target in test_2025.columns and test_2025[target].notna().any()
                if actual_available:
                    actual_top10 = test_2025.dropna(subset=[target]).nlargest(10, target)['title'].tolist()
                    overlap = len(set(actual_top10) & set(predicted_top10))
                    recall_at_10 = overlap / max(len(actual_top10), 1)
                    false_negatives = sorted(set(actual_top10) - set(predicted_top10))
                    false_positives = sorted(set(predicted_top10) - set(actual_top10))
                    print(f'Recall@10: {recall_at_10:.2%}')
                    if false_negatives:
                        print('Missed (actual top 10 not predicted):', false_negatives)
                    if false_positives:
                        print('False positives (predicted but not actual):', false_positives)
                    print(f'Overlap with actual 2025 top 10: {overlap}/10')
                else:
                    print('Actual 2025 revenues unavailable, skipping overlap metrics.')
            else:
                print('Unable to compute top 10 predictions for 2025.')


📁 Loading dataset: ../data/dataset_domestic_processed_english_2015_2026.csv
No title corrections needed
   ✅ Loaded 1,307 movies
   Year range: 2015-2026
   Training (≤2023): 1,003 movies
   Testing (2024): 145 movies
   Evaluation (2025): 114 movies
   Prediction (2026): 45 movies
Loaded dataset: ../data/dataset_domestic_processed_english_2015_2026.csv (1307 rows)
   Feature columns: 68
   Target column: revenue_domestic


2025 evaluation snapshot:
   RMSE: $53,612,383
   MAE:  $25,392,629
   MAPE: 212.8%
   R^2:  0.589
   Recall@10: 70.00%
   Precision@10: 70.00%
   NDCG@10: 0.821


,title,predicted_revenue,actual_revenue,prediction_error_pct,is_marvel,is_dc,is_star_wars,is_superhero,is_sequel,is_live_action_remake,is_major_studio,is_disney,is_english,is_origin_usa,is_origin_uk_ie,is_origin_canada,is_origin_us_uk_ca
0,Superman,"229,360,294.16","354,136,363.00",35.23,0,1,0,1,0,0,1,0,1,1,0,0,1
1,Thunderbolts*,"226,719,951.04","190,274,328.00",-19.15,1,0,0,1,0,0,1,1,1,1,0,0,1
2,Jurassic World Rebirth,"223,191,714.48","339,640,400.00",34.29,0,0,0,0,0,0,1,0,1,1,0,0,1
3,Captain America: Brave New World,"200,171,337.93","200,500,001.00",0.16,1,0,0,1,1,0,1,1,1,1,0,0,1
4,How to Train Your Dragon,"185,392,673.56","262,958,100.00",29.50,0,0,0,0,0,1,1,0,1,1,0,0,1
5,Lilo & Stitch,"163,438,323.77","423,773,167.00",61.43,0,0,0,0,0,1,1,1,1,1,0,0,1
6,Snow White,"163,223,734.45","87,203,963.00",-87.17,0,0,0,0,0,1,1,1,1,1,0,0,1
7,A Minecraft Movie,"130,487,869.01","423,949,195.00",69.22,0,0,0,0,0,0,1,0,1,1,0,0,1
8,28 Years Later,"113,414,137.64","70,446,897.00",-60.99,0,0,0,0,1,0,1,0,1,1,1,0,1
9,Elio,"106,005,421.57","72,987,454.00",-45.24,0,0,0,0,0,0,1,1,1,1,0,0,1


Recall@10: 70.00%
Missed (actual top 10 not predicted): ['Mission: Impossible - The Final Reckoning', 'Sinners', 'The Conjuring: Last Rites']
False positives (predicted but not actual): ['28 Years Later', 'Elio', 'Snow White']
Overlap with actual 2025 top 10: 7/10
